## Video Planning (Chain of Judge)

In diesem Projekt greifen wir auf ein neues Konzept zurück, um die Qualität und Effizienz von Lösungen zu verbessern. Wir nutzen zwei Sprachmodelle, einen Generator und einen Critic, um eine Feedback-Schleife zu bilden. Diese Methode basiert auf der sogenannten Self-Consistency mit Feedback-Schleife oder Critic-and-Generator-Framework, die es ermöglicht, sowohl die Qualität der Lösung als auch ihre Erklärbarkeit zu erhöhen.


Der Generator-Modell erstellt eine Lösung und gibt dabei durch Chain-of-Thought Prompting eine schrittweise Erklärung ab. Der Critic-Modell bewertet den Vorschlag des Generators und identifiziert mögliche Schwächen oder Fehler. Dieser Prozess wiederholt sich in einer Feedback-Schleife, wodurch der Generator seine Lösung kontinuierlich verbessern kann.


Diese innovative Methode bietet mehrere Vorteile: Sie ermöglicht eine Qualitätskontrolle durch den Critic, schrittweise Optimierung des Generators und erhöhte Erklärbarkeit durch Chain-of-Thought-Prompting. Darüber hinaus ist sie flexibel und anpassbar für verschiedene Aufgaben und Anwendungsfälle.


In den folgenden Abschnitten wird aufgezeigt, wie diese Methode umgesetzt wird und welche Vorteile sie bietet. Wir werden uns auf die Implementierung des Generators und Critic-Modells konzentrieren, sowie auf die Ergebnisse der Feedback-Schleife und die daraus gewonnenen Erkenntnisse.

### Aufgabe

Zeichne den Punkt P(4|6) in ein Koordinatensystem ein.

### Soll Lösung

1. Zeichne ein Koordinatensystem mit einer x-Achse und einer y-Achse, damit beide der Punkt im Koordinatensystem liegt <br>
		Achte darauf, die Achsen zu beschriften und eine regelmäßige Skalierung zu verwenden. <br>
2. Markiere den Ursprung (0, 0) des Koordinatensystems deutlich.<br>
3. Zeichne einen Vektor in x-Richtung, der von (0, 0) bis zum X Punkt reicht (x,  0). <br>
		Nutze eine klare Farbe wie Rot, um diesen Vektor zu kennzeichnen. <br>
4. Zeichne einen Vektor in y-Richtung, der von (0, 0) bis zum Y Punkt reicht (0, y). <br>
		Verwende eine andere Farbe, z. B. Grün. <br>
5. Verschiebe den y-Richtungs-Vektor so, dass sein Startpunkt das Ende des x-Richtungs-Vektors bei (x, 0) ist. <br>
6. Der Endpunkt dieses verschobenen Vektors markiert die Koordinaten (x, y).  <br>
		Zeichne einen Punkt an dieser Stelle und beschrifte ihn mit 'P({x-Wert}|{y-Wert})'. <br>
7. Füge ggf. Beschriftungen oder zusätzliche Hinweise hinzu, um die Schritte der Konstruktion verständlicher zu machen. <br>

In [1]:
# OpenAI Client für Ollama
from openai import OpenAI

# VectorDB
import chromadb
from sentence_transformers import SentenceTransformer

# Utility
import json

In [2]:
# ChromaDB initialisieren
chroma_client = chromadb.PersistentClient(path="vectordb")

# Sentence Transformer initialisieren
dimensions = 1024
model = SentenceTransformer(
    "mixedbread-ai/deepset-mxbai-embed-de-large-v1", truncate_dim=dimensions
)

# Verbindung zum Ollama-Server herstellen
client = OpenAI(
    base_url="http://localhost:11434/v1",
    api_key="ollama",
)

### Collection laden oder initialisieren

Wenn die Collection noch nicht existiert, wird sie hier einmalig und persistent erstellt und mit Daten gefüllt.

In [3]:
# Check for existing collections
existing_collections = chroma_client.list_collections()
existing_collections

['serlo_collection']

In [4]:
# Serlo Collection von persistierter ChromaDB laden
collection_name = "serlo_collection"

if collection_name not in existing_collections:
    serlo_data = {}

    with open("Experiments/RAG/serlo_data.json", "r") as f:
        serlo_data = json.load(f)

    collection = chroma_client.create_collection(collection_name)

    # Add content chunks to vector database
    for i, article_url in enumerate(serlo_data["article_urls"]):
        content_chunks = serlo_data["content_chunks"][i]

        embeddings = model.encode(content_chunks)

        collection.add(
            documents=content_chunks,
            embeddings=embeddings,
            metadatas=[{"source": article_url} for _ in content_chunks],
            ids=[
                str(article_url.split("/")[-1]) + "_" + str(i)
                for i, _ in enumerate(content_chunks)
            ],
        )
else:
    collection = chroma_client.get_collection(collection_name)

In [5]:
def retrieve_additional_context(task_description, n_results=3):
    query_embedding = model.encode([task_description])

    # Retrieval Schritt
    results = collection.query(
        query_embeddings=query_embedding, n_results=n_results, include=["documents"]
    )

    for documents in results["documents"]:
        context = "Kontext:\n"
        context += "\n\nKontext:\n".join(documents)

    return context

In [6]:
# Generator-Modell
def generate_visual_steps(task_description, use_rag=False):
    response = client.chat.completions.create(
        model="phi4",
        messages=[
            {
                "role": "system",
                "content": (
                    "Du bist ein Sprachmodell, das Mathematikaufgaben in präzise, nummerierte Schritte zerlegt, "
                    "die leicht visuell dargestellt werden können. Gib die Antwort als eine Schritt-für-Schritt-Liste zurück, "
                    "beginnend mit '1.', '2.', usw. Jede Nummer sollte einen klar definierten Schritt enthalten."
                ),
            },
            {
                "role": "user",
                "content": retrieve_additional_context(task_description, n_results=3)
                + "\n\nAufgabenbeschreibung: "
                + task_description
                if use_rag
                else task_description,
            },
        ],
    )
    return response.choices[0].message.content

In [7]:
# Bewertungsmodell
def evaluate_solution(task_description, generated_solution):
    response = client.chat.completions.create(
        model="phi4",
        messages=[
            {
                "role": "system",
                "content": (
                    "Du bist ein Sprachmodell, das Lösungen für Mathematikaufgaben bewertet. "
                    "Bewerte die Lösung nicht nur auf Korrektheit, sondern auch darauf, ob sie die folgenden Kriterien erfüllt:\n"
                    "1. Die Lösung muss als nummerierte Schritt-für-Schritt-Anleitung vorliegen.\n"
                    "2. Die Schritte müssen klar und präzise formuliert sein.\n"
                    "3. Begriffe wie 'Koordinatensystem', 'x-Achse', 'y-Achse', 'Vektor' sollten verwendet werden, "
                    "wenn sie für die Aufgabe relevant sind.\n"
                    "Antworte mit einer kurzen Bewertung und sage klar, ob die Lösung akzeptiert wird oder nicht."
                ),
            },
            {
                "role": "user",
                "content": f"Aufgabe:\n{task_description}\n\nLösung:\n{generated_solution}",
            },
        ],
    )
    return response.choices[0].message.content

In [8]:
# Chain-of-Judge
def interactive_solution(task_description, max_iterations=2, use_rag=False):
    iteration = 0
    solution = generate_visual_steps(task_description, use_rag)
    final_solution = None
    print(f"Iteration {iteration + 1} - Generierte Lösung:\n{solution}")

    while iteration < max_iterations:
        evaluation = evaluate_solution(task_description, solution)
        print(f"Iteration {iteration + 1} - Bewertung der Lösung:\n{evaluation}")

        if "akzeptiert" in evaluation.lower() and "nummeriert" in evaluation.lower():
            print("Lösung akzeptiert!")
            final_solution = solution
            break
        else:
            iteration += 1
            if iteration < max_iterations:
                print(f"Neue Iteration {iteration + 1}: Verbesserung der Lösung...")
                solution = generate_visual_steps(task_description, use_rag)
                print(
                    f"Iteration {iteration + 1} - Neue generierte Lösung:\n{solution}"
                )
            else:
                print(
                    "Maximale Anzahl an Iterationen erreicht. Keine weiteren Verbesserungen möglich."
                )

    if final_solution is None:
        final_solution = solution
    return final_solution

### Videoplan generieren

In der Theorie kann die Pipeline mit jeder möglichen Matheaufgabe oder Thema verwendet werden. Praktisch werden Aufgaben mit Bezug zur Geometrie wohl bessere Ergebnisse liefern. Einige mögliche Aufgaben finden sich hier: https://de.serlo.org/mathe/1288/geometrie

In [9]:
# Beispielaufgaben
task_description = "Zeichne das Dreieck mit den Eckpunkten A(1|1), B(5|1), und C(3|4) in ein Koordinatensystem. Beschrifte die Punkte und verbinde sie." # Am meisten getestet
# task_description = "Zeichne den Punkt P(4|6) in ein Koordinatensystem ein."
# task_description = "Zeichne die Gerade durch die Punkte A(1|2) und B(4|5) in ein Koordinatensystem."

# Ablauf mit maximal 2 Iterationen starten
final_solution = interactive_solution(task_description, max_iterations=2, use_rag=True)

Iteration 1 - Generierte Lösung:
Um das Dreieck mit den Eckpunkten \( A(1, 1) \), \( B(5, 1) \), und \( C(3, 4) \) in ein Koordinatensystem zu zeichnen, folgen Sie diesen detaillierten Schritten:

1. **Stellen des Koordinatensystems auf**:  
   - Zeichnen Sie zwei Perpendikel schneidende Achsen: eine vertikale (y-Achse) und eine horizontale (x-Achse).
   - Fügen Sie Pfeile bei jedem Ende der Achsen an, um die positive Richtung zu zeigen. Die y-Achse zeigt nach oben und die x-Achse zeigt nach rechts.

2. **Beschriften der Achsen**:  
   - Beschriften Sie die y-Achse mit "yyy".
   - Beschriften Sie die x-Achse mit "xxx".

3. **Einheiten markieren**:
   - Wählen Sie eine Skalierung, bei der zwei Kästchen (oder Einheiten) eine Zeile oder Spalte ausmachen.
   - Verwenden Sie gerade Striche an jeder Achse, um die Einheiten zu markieren.

4. **Setzen des Ursprungs**:  
   - Kennzeichnen Sie den Schnittpunkt der x- und y-Achsen als \( O \).

5. **Einzeln die Punkte einzeichnen**:
   - Erstelle

In [10]:
final_solution

'Um das Dreieck mit den Eckpunkten \\( A(1, 1) \\), \\( B(5, 1) \\), und \\( C(3, 4) \\) in ein Koordinatensystem zu zeichnen, folgen Sie diesen detaillierten Schritten:\n\n1. **Stellen des Koordinatensystems auf**:  \n   - Zeichnen Sie zwei Perpendikel schneidende Achsen: eine vertikale (y-Achse) und eine horizontale (x-Achse).\n   - Fügen Sie Pfeile bei jedem Ende der Achsen an, um die positive Richtung zu zeigen. Die y-Achse zeigt nach oben und die x-Achse zeigt nach rechts.\n\n2. **Beschriften der Achsen**:  \n   - Beschriften Sie die y-Achse mit "yyy".\n   - Beschriften Sie die x-Achse mit "xxx".\n\n3. **Einheiten markieren**:\n   - Wählen Sie eine Skalierung, bei der zwei Kästchen (oder Einheiten) eine Zeile oder Spalte ausmachen.\n   - Verwenden Sie gerade Striche an jeder Achse, um die Einheiten zu markieren.\n\n4. **Setzen des Ursprungs**:  \n   - Kennzeichnen Sie den Schnittpunkt der x- und y-Achsen als \\( O \\).\n\n5. **Einzeln die Punkte einzeichnen**:\n   - Erstellen Sie 

### Videoplan speichern

In [11]:
# Übersetzungsmodell
def translate_solution(generated_solution):
    response = client.chat.completions.create(
        model="phi4",
        messages=[
            {
                "role": "system",
                "content": (
                    "Du bist ein Sprachmodell, das Videopläne ins Englische übersetzt."
                    "Übersetze nur den dir gegebenen Text und füge keine eigenen Informationen hinzu."
                ),
            },
            {
                "role": "user",
                "content": f"Videoplan:\n{generated_solution}\n\nÜbersetzung:",
            },
        ],
    )
    return response.choices[0].message.content

In [12]:
# Nach Englisch übersetzen
translated_solution = translate_solution(final_solution)
translated_solution

'Video Plan:\nUm das Dreieck mit den Punkten \\( A(1, 1) \\), \\( B(5, 1) \\), und \\( C(3, 4) \\) in ein Koordinatensystem zu zeichnen, folgen Sie diesen detaillierten Schritten:\n\n1. **Koordinatensystem aufstellen**:  \n   - Zeichnen Sie zwei senkrecht zueinander stehende Achsen: eine vertikale (y-Achse) und eine horizontale (x-Achse).\n   - Setzen Sie Pfeile an den Enden der Achsen, um die positive Richtung zu zeigen. Bei der y-Achse zeigt der Pfeil nach oben und bei der x-Achse nach rechts.\n\n2. **Achsen beschriften**:  \n   - Beschriften Sie die y-Achse mit "yyy".\n   - Beschriften Sie die x-Achse mit "xxx".\n\n3. **Einheiten markieren**:\n   - Wählen Sie eine Skalierung, bei der zwei Kästchen (oder Einheiten) eine Zeile oder Spalte ausmachen.\n   - Verwenden Sie gerade Striche an jeder Achse, um die Einheiten zu markieren.\n\n4. **Ursprung setzen**:  \n   - Kennzeichnen Sie den Schnittpunkt der x- und y-Achsen als \\( O \\).\n\n5. **Einzelne Punkte eintragen**:\n   - Erstellen 

In [13]:
part_1_results = {
    "task_description": task_description,
    "video_plan": translated_solution
}

In [14]:
with open("results/part_1_results.json", "w") as f:
    json.dump(part_1_results, f)